#ChatBot Development (Part 2)

## This is a basic notebook with everything to design and experipment with a ChatBot. You have the ChatBot code, a simple UI and a logging file for each dialogue session. The main objective for the assignment is to design a ChatBot by:
*   Selecting an application
*   Designing the ChatBot Prompt
*   Experimenting and evaluating
*   Keeping the best dialogue sessions / examples


In [ ]:
%%capture
!pip install langchain-openai openai

In [ ]:
import os
import openai

In [ ]:
from openai.types import Completion, CompletionChoice, CompletionUsage

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnableMap,RunnableSequence
import time

# Set up OpenAI API key
import os
os.environ["OPENAI_API_KEY"] =

# Prompt definition

### The ChatBot's behaviour is defined through a specialised Prompt. In `LangChain` you can use prompt `Templates` that make it possoble to define complex Prompts, including variables and call them as part of a 'chain'.

In [ ]:
# Define the prompt template
# I have included high-level descriptions of the things you
# will need to write to create a functional Prompt (this one does NOT work as such!!!)
#
#
prompt_template = PromptTemplate.from_template("""
<Persona>
You are a confident, witty, and slightly cheeky football fan who loves a good debate.
You have strong opinions, but you respect others. You enjoy banter and can push back if someone says something wild.
You know your football — stats, tactics, legends, and drama — and you’re not afraid to use it in a debate.
You sound like someone talking in a football pub, on a group chat, or during half-time at a match.
Your reply Should be at the same momentum with user
</Persona>

<Task>
This dialogue is about football: including but not limited to player stats, match analysis, transfer news, historical comparisons, tactical breakdowns, and fan debates.

Objectives:
- Provide clear and engaging information about football topics the user brings up.
- Educate the user when needed with relevant facts or breakdowns.
- Engage in thoughtful conversation and opinionated takes when appropriate.
- Ask follow-up questions to keep the dialogue dynamic and interactive.

Dialogue Strategies:
- Use retrieval-based facts for stats, match results, and player performances.
- Apply Zero-Shot Chain of Thought reasoning for complex user questions (e.g., comparing legends or simulating match outcomes).
- Use analogies and visuals (if applicable) to explain tactical setups or formations.
- Ask for the user’s opinion to maintain engagement.
- Begin with confident, insightful analysis when answering football questions.
- When the user brings in a hot take or wild opinion, switch tone to cheeky or banter-heavy.
- Use phrases like “nahhh, behave 😂” or “you can’t be serious!” to reflect banter mode.
- After a few cheeky lines, smoothly return to thoughtful analysis to balance the tone.
- Always read the energy of the user — if they’re debating, match the intensity; if they’re joking, play along.
- If the user deviates from football, respond playfully but steer the conversation back to the core topic.
- Acknowledge the fun moment, but remind the user of the ongoing discussion or question.
- Use smooth transitions like “Speaking of which…” or “Now, back to the pitch…” to regain focus.

</Task >

<Communication>
-Keep your tone playful, opinionated, and ready to argue — but never rude.
-Don't be afraid to challenge the user’s opinion or play devil’s advocate.
-Use punchy replies, rhetorical questions, or comparisons to make your point.
-Throw in occasional football slang (like "bottle it", "top bin", "fraud watch", "generational", etc.) when it fits.
-If the user says something you disagree with, call them out — respectfully — and ask them to back it up.
-Encourage the user to defend their take or respond with theirs, so the chat feels like a true back-and-forth debate.
-Keep your responses short-to-medium unless the user wants deep analysis.
-Fallback Strategy:
-Let the argument be a strong point of view.
-If user input is vague, ask for clarification or suggest a few football topics.
</Communication>

Conversation history:
{history}

User: {user_input}
Sportbot:
""")

# This is the core of LLM invocation with LangChain. You use `ChatOpenAI` instead of the usual completion function

more documentation here:
https://python.langchain.com/docs/integrations/chat/openai/

In [ ]:
# Initialize memory with a max token limit
# a dialogue needs memory; the bot needs to remember
# what was said before or what was previously asked
# you can make a realistic bot by having a limited memory, like a human
#
memory = ConversationBufferMemory(
    return_messages=True,
    max_token_limit=500  # Keeps the buffer concise
)

# Initialize the chat model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.8)

# Create the conversation chain using RunnableSequence
# This is a pipeline approach (as you have in R or in Unix)
chain = prompt_template | llm

## This is the UI part. You can replace it with your own for the assignment, or keep or modify this one.

# Now you should be ready for your project / assignment

## You can use the base Notebooks, or modify them and incorporate other aspects, such as RAG, or change the User Interface.

### Hint: in LangChain you can have RAG via RetrievalQA and use FAISS as per the previous Parctical

## The next four sessions (Mondays 24, 31 and Fridays 28, 4) will be entirely dedicated to developing your ChatBot.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import textwrap
from datetime import datetime

# Dummy chatbot logic (replace with LangChain logic)
def chatbot_response(user_input):
    try:
        # Simulated response (replace with your actual LangChain chain.invoke logic)
        response_text = f"This is a response to: {user_input}"
        return response_text
    except Exception as e:
        return f"Error: {str(e)}"

# Setup chat output with scrollable area
chat_display = widgets.Output()
chat_display.layout = widgets.Layout(height='400px', overflow='auto')

# Input field and buttons
user_input = widgets.Textarea(
    placeholder="Type your message...",
    layout=widgets.Layout(width="100%", height="50px")
)
send_button = widgets.Button(description="Send", button_style="primary")
end_button = widgets.Button(description="End Chat", button_style="danger")
button_box = widgets.HBox([send_button, end_button])
chat_ui = widgets.VBox([chat_display, user_input, button_box])

# Display UI
display(chat_ui)

# Open log file
log_filename = "chat_history.txt"
log_file = open(log_filename, "a", encoding="utf-8")

# Conversation history
history = []

def update_chat(user, bot):
    wrapped_user = textwrap.fill(user, width=100)
    html_user = wrapped_user.replace('\n', '<br>')

    wrapped_bot = textwrap.fill(bot, width=100)
    html_bot = wrapped_bot.replace('\n', '<br>')

    timestamp = datetime.now().strftime('%H:%M:%S')

    chat_display.append_display_data(HTML(f"""
        <div style='margin-top: 12px; padding: 5px; border-bottom: 1px solid #ddd;'>
            <div><strong style='color:#1a73e8;'>User [{timestamp}]:</strong> {html_user}</div>
            <div><strong style='color:#34a853;'>ChatBot:</strong> {html_bot}</div>
        </div>
    """))


def handle_input(_=None):
    user_text = user_input.value.strip()
    if not user_text:
        return

    if user_text.lower() in ["exit", "quit"]:
        stop_chat()
        return

    bot_response = chatbot_response(user_text)
    update_chat(user_text, bot_response)

    history.append({"user": user_text, "bot": bot_response})
    log_file.write(f"User: {user_text}\nChatBot: {bot_response}\n\n")
    log_file.flush()

    user_input.value = ""

def stop_chat(_=None):
    global log_file
    log_file.close()
    update_chat("Chat Ended", "Chat history has been saved.")
    user_input.disabled = True
    send_button.disabled = True
    end_button.disabled = True

# Bind buttons to actions
send_button.on_click(handle_input)
end_button.on_click(stop_chat)

def chatbot_response(user_input):
    try:
        response = chain.invoke({
            "history": history,  # if you're passing memory
            "user_input": user_input
 # or change to "user_input" depending on your prompt template
        })
        return response.content  # or response['output'] depending on your setup
    except Exception as e:
        return f"Error: {str(e)}"
